<a href="https://colab.research.google.com/github/pranshurastogi29/Miner_fees_optimization_RL/blob/main/fees_optimization_using_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install stable-baselines3[extra]

In [15]:
import math
import numpy as np
import pandas as pd
import gym
from gym import spaces
from gym.utils import seeding

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3 import DQN

class FeesMaximizationEnv(gym.Env):
    """
    Description:
        The agent (a miner) want to select the transaction. For any given
        state the agent may choose to accept or reject the transaction keeping 
        weight is less then limit
    
    Observation:
        Type: Box(2)
        Num    Observation               Min            Max
        0      Fee                       0              inf
        1      Weights                   0              4000000
    Actions:
        Type: Discrete(3)
        Num    Action
        0      Dont process the transaction
        1      Process the transaction

    Reward:
         Reward of 1 is awarded if the running average fee increases
         Reward of -1 is awarded if the running average fee decreases

    Episode Termination:
         Weight exceeds 4000000
    """

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 30
    }

    def __init__(self, goal_velocity=0):
        self.min_fee = 0
        self.max_fee =  np.finfo(np.float32).max
        self.max_weight = 4000000
        self.min_weight = 0
        self.df = pd.read_csv('mempool.csv')[['tx_id','fee','weight']]
        self.low = np.array(
            [self.min_fee, self.min_weight], dtype=np.float32
        )
        self.high = np.array(
            [self.max_fee, self.max_weight], dtype=np.float32
        )

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(
            self.low, self.high, dtype=np.float32
        )

        self.seed()
        self.current_fee = 0
        self.current_weight = 0
        self.c = 1
        self.select_hash = []

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def select_transaction(self):
        seed = np.random.choice(np.arange(len(self.df)), 1, replace=False)[0]
        return self.df.values[seed]

    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid" % (action, type(action))

        fee , weight  = self.state
        select_hash , selected_fee , selected_weight = self.select_transaction()

        if (fee + selected_fee) > (self.current_fee / self.c):
          self.current_fee  = fee + selected_fee
          self.current_weight = weight + selected_weight
          self.c = self.c + 0.5
          self.select_hash.append(select_hash)
          reward = 1

        elif (fee + selected_fee) <= (self.current_fee / self.c):
          reward = -1


        done = bool(
          self.current_weight > self.max_weight
        )

        self.state = (self.current_fee, self.current_weight)
        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = np.array([0,0])
        self.current_fee, self.current_weight = 0 , 0
        self.c = 1
        self.select_hash = []
        return np.array(self.state)

Writing env.py


In [21]:
env=FeesMaximizationEnv()

In [22]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(len(env.select_hash))
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

1972
Episode:1 Score:1972
2119
Episode:2 Score:2119
2172
Episode:3 Score:2172
2099
Episode:4 Score:2099
2057
Episode:5 Score:2057


In [5]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [6]:
model.learn(total_timesteps=200000)

-----------------------------
| time/              |      |
|    fps             | 746  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 638        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01267374 |
|    clip_fraction        | 0.0598     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.68      |
|    explained_variance   | 0.0505     |
|    learning_rate        | 0.0003     |
|    loss                 | 13.8       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.00202   |
|    value_loss           | 98.9       |
----------------------------------------
------------------------------------------
| time/  

In [8]:
obs = env.reset()
l = []
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    l.append(env.envs[0].select_hash)
    if done: 
        print('info', info)
        break
selected_hash_PPO = l[-2]
Optimized_fees_PPO , Optimized_weight  = info[0]['terminal_observation']
env.close()

info [{'terminal_observation': array([3444175, 4000584])}]


In [9]:
textfile = open("Fees_optimied_PPO.txt", "a+")

for element in selected_hash_PPO:

    textfile.write(element + "\n")

textfile.close()

In [10]:
model = DQN('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=200000)


Using cpu device
----------------------------------------
| rollout/                |            |
|    exploration rate     | 0.607      |
| time/                   |            |
|    episodes             | 4          |
|    fps                  | 1407       |
|    time_elapsed         | 5          |
|    total timesteps      | 8281       |
| train/                  |            |
|    approx_kl            | 0.00918376 |
|    clip_fraction        | 0.0775     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.654     |
|    explained_variance   | -1.19e-07  |
|    learning_rate        | 0.0003     |
|    loss                 | 24         |
|    n_updates            | 980        |
|    policy_gradient_loss | -0.00181   |
|    value_loss           | 38.4       |
----------------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.21     |
| time/               |          |
|    episodes         |

In [11]:
obs = env.reset()
c = []
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    c.append(env.envs[0].select_hash)
    if done: 
        print('info', info)
        break
selected_hash_DQN = c[-2]
Optimized_fees_DQN , Optimized_weight_DQN  = info[0]['terminal_observation']
env.close()

info [{'terminal_observation': array([3096998, 4011052])}]


In [12]:
textfile = open("Fees_optimied_DQN.txt", "a+")

for element in selected_hash_DQN:

    textfile.write(element + "\n")

textfile.close()